# Machine learning with TensorFlow: two hidden layers

Date: August 21-23, 2018

Neural network: two hidden layers

Classification of human body motion:
- walking
- sitting down
- turning right while walking
- turning left while walking

In [1085]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sg
import serial
import time
import random

## Preparing data set for training CNN

In [1086]:
SAMPLE_NUMS = 260  # per sec
SAMPLING_RATE = 80.0  # Hz
GYRO_RESOLUTION = 250.0 / 32768.0
ACCEL_RESOLUTION = 2.0 / 32768.0

TRAINING_SET = [['./20180821_walking_straight.csv', 0],
               ['./20180821_sitting.csv', 1],
               ['./20180821_turning_left.csv', 2],
               ['./20180821_turning_right.csv', 3],                
               ['./20180823_walking_straight.csv', 0],
               ['./20180823_sitting.csv', 1],
               ['./20180823_turning_left.csv', 2],
               ['./20180823_turning_right.csv', 3]]

TEST_SET = [['./20180822_walking_straight.csv', 0],
            ['./20180822_sitting.csv', 1],
            ['./20180822_turning_left.csv', 2],
            ['./20180822_turning_right.csv', 3]]

TIME_INTERVAL = 260.0 / SAMPLING_RATE  # sec

MEASUREMENTS = 8
RECORDS = 260

In [1087]:
gres = lambda v: v * GYRO_RESOLUTION
ares = lambda v: v * ACCEL_RESOLUTION
to_time = lambda v: v / SAMPLING_RATE

def conv(df, lpf=False):
    df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(gres)
    df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(ares)
    if lpf:
        CUTOFF = 10.0
        b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
        df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(lambda row: sg.lfilter(b, a, row))
        CUTOFF = 10.0
        b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
        df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(lambda row: sg.lfilter(b, a, row))
    df[['cnt']] = df[['cnt']].apply(to_time)
    df.set_index('cnt', drop=True, inplace=True)
    # measurements = df.tail(1).iloc[0,0] + 1
    # return measurements

In [1088]:
training_set = []
test_set = []

def label(l):
    ll = [0,0,0,0]
    ll[l] = 1
    return ll

for file, l in TRAINING_SET:
    df = pd.read_csv(file, dtype=np.int16)
    conv(df, lpf=True)
    for i in range(MEASUREMENTS):
        training_set.append([df[df['id']==i], label(l)])
        
for file, l in TEST_SET:
    df = pd.read_csv(file, dtype=np.int16)
    conv(df, lpf=True)
    for i in range(MEASUREMENTS):
        test_set.append([df[df['id']==i], label(l)])
    
random.shuffle(training_set)
random.shuffle(test_set)

In [1089]:
# use accel x axis and gyro z axis values for classification of human body motion
train_x, train_t = [], []
test_x, test_t = [], []
for df, label in training_set:
    values = np.concatenate((df['ax'].values, df['ay'].values, df['az'].values,
                             df['gx'].values, df['gy'].values, df['gz'].values),
                            axis=None)
    train_x.append(values)
    train_t.append(label)
for df, label in test_set:
    values = np.concatenate((df['ax'].values, df['ay'].values, df['az'].values,
                             df['gx'].values, df['gy'].values, df['gz'].values),
                            axis=None)
    test_x.append(values)
    test_t.append(label)

## TensorFlow

In [1090]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20180823)
tf.set_random_seed(20180823)

In [1091]:
num_samples = SAMPLE_NUMS * 6
num_units1 = 64
num_units2 = 64
num_classes = 4

x = tf.placeholder(tf.float32, [None, num_samples])

w1 = tf.Variable(tf.truncated_normal([num_samples, num_units1]))
b1 = tf.Variable(tf.zeros([num_units1]))
hidden1 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.zeros([num_units2]))
hidden2 = tf.nn.sigmoid(tf.matmul(hidden1, w2) + b2)

w0 = tf.Variable(tf.zeros([num_units2, num_classes]))
b0 = tf.Variable(tf.zeros([num_classes]))
p = tf.nn.softmax(tf.matmul(hidden2, w0) + b0)

In [1092]:
t = tf.placeholder(tf.float32, [None, num_classify])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [1093]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [1094]:
i = 0
for _ in range(2000):
    i += 1
    sess.run(train_step, feed_dict={x:train_x, t:train_t})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:test_x, t:test_t})
        print('Step: {}, Loss: {}, Accuracy: {}'.format(i, loss_val, acc_val))

Step: 100, Loss: 18.577682495117188, Accuracy: 0.96875
Step: 200, Loss: 8.740309715270996, Accuracy: 0.96875
Step: 300, Loss: 6.4966278076171875, Accuracy: 0.96875
Step: 400, Loss: 5.67677116394043, Accuracy: 0.96875
Step: 500, Loss: 5.228507041931152, Accuracy: 0.96875
Step: 600, Loss: 4.886094093322754, Accuracy: 0.96875
Step: 700, Loss: 4.787983417510986, Accuracy: 0.96875
Step: 800, Loss: 4.74266242980957, Accuracy: 0.96875
Step: 900, Loss: 4.723090648651123, Accuracy: 0.96875
Step: 1000, Loss: 4.714951992034912, Accuracy: 0.96875
Step: 1100, Loss: 4.70755672454834, Accuracy: 0.96875
Step: 1200, Loss: 4.704200744628906, Accuracy: 0.96875
Step: 1300, Loss: 4.692885875701904, Accuracy: 0.96875
Step: 1400, Loss: 4.686490058898926, Accuracy: 0.96875
Step: 1500, Loss: 4.686164379119873, Accuracy: 0.96875
Step: 1600, Loss: 4.688432216644287, Accuracy: 0.96875
Step: 1700, Loss: 4.690561294555664, Accuracy: 0.96875
Step: 1800, Loss: 4.7003021240234375, Accuracy: 0.96875
Step: 1900, Loss: 4

In [1095]:
p_test = sess.run(p, feed_dict={x:test_x})
(p_test*100).astype(int)[:10]

array([[ 7,  1, 90,  0],
       [85,  0, 13,  0],
       [98,  0,  1,  0],
       [ 0, 99,  0,  0],
       [ 1,  0, 98,  0],
       [97,  0,  2,  0],
       [ 0, 99,  0,  0],
       [ 0, 99,  0,  0],
       [ 0,  0,  0, 99],
       [ 0,  0, 99,  0]])

In [1096]:
test_t[:10]

[[1, 0, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0]]

In [1097]:
sess.close()